In [18]:
import numpy as np
import pandas as pd
import geopandas as gpd
import rasterio as rio
from rasterio import features
from glob import glob
from tqdm import tqdm
from PIL import Image
import os

In [29]:
states = ["ak"]
states_group = "-".join(states)

celltowers = gpd.read_file("celltowers/shapefiles/CellularTowers.shp")
cells = celltowers[celltowers["LOCSTATE"].isin(map(str.upper,states))]

open_path = glob("sentinel2/{}/composite/*.tif".format(states_group))
save_path = "celltowers/sentinel2/{}/".format(states_group)

In [30]:
def cellraster_from_sat(cells, path):
    with rio.open(path) as dataset:

        satimg = dataset.read()
        mask = dataset.dataset_mask()
        sat_trs = dataset.transform
        sat_crs = dataset.crs

        results = (
            {'properties': {'raster_val': v}, 'geometry': s}
            for i, (s, v) 
            in enumerate(
                rio.features.shapes(mask, transform=dataset.transform)))

    sat = gpd.GeoDataFrame.from_features(list(results),crs=sat_crs)
    cells = cells.to_crs(sat_crs)
    intersects = cells[cells.intersects(sat.unary_union)]
    cells_array = [(geom.buffer(400,20), 1) for geom in intersects["geometry"]]
    cells_raster = features.rasterize(cells_array,np.shape(satimg[0]),transform=sat_trs)
    return Image.fromarray(cells_raster)

In [31]:
for path in tqdm(open_path):
    sat_id = path.split("\\")[1].split(".")[0]
    if not os.path.isdir(save_path): os.makedirs(save_path)
    cellraster = cellraster_from_sat(cells, path)
    cellraster.save(save_path+sat_id+".jpg")

  0%|          | 0/19 [00:00<?, ?it/s]C:\Users\Tyler\anaconda3\envs\cs230\lib\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
  5%|▌         | 1/19 [00:03<00:59,  3.29s/it]C:\Users\Tyler\anaconda3\envs\cs230\lib\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
  5%|▌         | 1/19 [00:05<01:37,  5.43s/it]


ValueError: No valid geometry objects found for rasterize

In [ ]:
%pylab
%matplotlib inline
imshow(cellraster)